# Graph API

In this notebook we'll work to achieve milestone 1: the ability to read my personal OneNote files which are stored in my personal OneDrive account in Azure.

## Create App Registration and Authentication

The first step is to create an Azure App Registration.  The reason we do this is it gives our application an identity in Azure which we can use to grant permissions.  We do this in the Azure Portal.  I named mine "Danu Agent".

If this were a commercial app, I'd be using managed identity.  But for now for local experimentation I'm going to create a client secret and make a way to retrieve these credentials.

## Create config file

Create a directory `config` in the project root, and create `config/danu-config.json` there as shown below.  Note that this file is not in source control and will not be checked in:

`config/danu-config.json`:
```json
{
    "danu-identity":
    {
        "app_id": "app id goes here. Fill this in from your app registration you created.",
        "client_secret": "app secret goes here. Create a client secret and put it here."
    }
}
```

In [1]:
from danu.infra import LocalJsonSecretsProvider

In [2]:
secrets = LocalJsonSecretsProvider('../../../config/danu-config.json') # This is at the top level, same level as 'src'

In [3]:
app_id = secrets.get_secret('danu-identity.app_id')
client_secret = secrets.get_secret('danu-identity.client_secret')
tenant_id = secrets.get_secret('danu-identity.tenant_id')
print(f'app id {app_id} tenant={tenant_id} secret len={len(client_secret)}')

app id 52e677b3-b988-4573-b9d0-18cdb5c7c67b tenant=32dd5da8-160d-4bc8-9fcc-9c8ed28043ea secret len=40


We now have an identity for our application established.

## Graph authentication

Working through ideas in [Microsoft Graph authentication and authorization overview](https://learn.microsoft.com/en-us/graph/auth/).


In [4]:
from msal import ConfidentialClientApplication

In [5]:
authority = f'https://login.microsoftonline.com/{tenant_id}'
scopes = ['https://graph.microsoft.com/.default']

app = ConfidentialClientApplication(app_id, authority=authority, client_credential=client_secret)
result = app.acquire_token_for_client(scopes)

In [6]:
result['token_type'], result['access_token'][0:3] + '...'

('Bearer', 'eyJ...')

In [7]:
access_token = result['access_token']

In [8]:
# Use Graph API to access OneNote data
import requests
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get('https://graph.microsoft.com/v1.0/me/onenote/notebooks', headers=headers)

In [9]:
response, response.content

(<Response [400]>,
 b'{"error":{"code":"BadRequest","message":"/me request is only valid with delegated authentication flow.","innerError":{"date":"2023-10-14T23:35:39","request-id":"727c30e6-0ba0-46c5-960a-b8afa064b00c","client-request-id":"727c30e6-0ba0-46c5-960a-b8afa064b00c"}}}')

One way to handle this would be to use a web site that has a public endpoint for retrieving the credentials via a redirect.  But since we're in a notebook, let's use an `InteractiveBrowserCredential` from the `azure.identity` package.  This will open a browser window for us to authenticate and grant permissions to the application.

In [10]:
from azure.identity import InteractiveBrowserCredential
import requests

In [14]:
redirect_uri = 'http://localhost:59513' # Must add this in portal for app/authentication/platform configurations/redirect uris
credential = InteractiveBrowserCredential(client_id=app_id, tenant_id=tenant_id, redirect_uri=redirect_uri, client_secret=client_secret)
token = credential.get_token('https://graph.microsoft.com/.default', client_secret=client_secret)
access_token = token.token

InteractiveBrowserCredential.get_token failed: AADSTS7000218: The request body must contain the following parameter: 'client_assertion' or 'client_secret'.
Trace ID: 06c78d72-316c-4851-988a-c9453e4d1f00
Correlation ID: 6c854f8e-f515-469b-a4a8-ca6e822ed33a
Timestamp: 2023-10-14 23:41:33Z


ClientAuthenticationError: AADSTS7000218: The request body must contain the following parameter: 'client_assertion' or 'client_secret'.
Trace ID: 06c78d72-316c-4851-988a-c9453e4d1f00
Correlation ID: 6c854f8e-f515-469b-a4a8-ca6e822ed33a
Timestamp: 2023-10-14 23:41:33Z

I apologize for the confusion earlier. It seems that the InteractiveBrowserCredential does not support the use of a client secret. In this case, you can use the DeviceCodeCredential instead, which is designed for situations where you don't have a web server to receive the credentials.

Note that you must go into your application under Authentication/Advanced and enable **Allow public client flows**.

In [34]:
from azure.identity import DeviceCodeCredential
scope = 'User.Read,Notes.Read'
one_note_content = '/me/onenote/notebooks' # For a OneDrive for business account. See https://learn.microsoft.com/en-us/graph/integrate-with-onenote
#one_note_content = '/me/drive/root/onenote/notebooks' # For a personal OneDrive account
credential = DeviceCodeCredential(client_id=app_id, tenant_id=tenant_id, scope=scope)
token = credential.get_token(scope)
access_token = token.token

headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get('https://graph.microsoft.com/v1.0' + one_note_content, headers=headers)

To sign in, use a web browser to open the page https://microsoft.com/devicelogin and enter the code E6U67WR78 to authenticate.


In [35]:
response.status_code, response.content

(404,
 b'{"error":{"code":"30108","message":"OneDrive for Business for this user account cannot be retrieved.","innerError":{"date":"2023-10-15T01:02:14","request-id":"33bed514-656d-4e7c-b77f-f0df481dc69b","client-request-id":"33bed514-656d-4e7c-b77f-f0df481dc69b"}}}')

In [36]:
access_token='...'
headers = {'Authorization': f'Bearer {access_token}'}
response = requests.get('https://graph.microsoft.com/v1.0' + one_note_content, headers=headers)

In [37]:
response.status_code, response.content

(200,
 b'{"@odata.context":"https://graph.microsoft.com/v1.0/$metadata#users(\'richardlack%40hotmail.com\')/onenote/notebooks","value":[{"id":"0-D837C5F832A7D046!513","self":"https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!513","createdDateTime":"2016-04-28T15:23:06.233Z","displayName":"Argon Collab","lastModifiedDateTime":"2017-11-04T16:33:20.203Z","isDefault":false,"userRole":"Owner","isShared":true,"sectionsUrl":"https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!513/sections","sectionGroupsUrl":"https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!513/sectionGroups","createdBy":{"user":{"id":"D837C5F832A7D046","displayName":"Richard Lack"}},"lastModifiedBy":{"user":{"id":"D837C5F832A7D046","displayName":"Richard Lack"}},"links":{"oneNoteClientUrl":{"href":"onenote:https://d.docs.live.net/d837c5f832a7d046/Documents/Argon%20Collab"},"oneN

---

The redirect URI 'http://localhost:59436' specified in the request does not match the redirect URIs configured for the application '52e677b3-b988-4573-b9d0-18cdb5c7c67b'. Make sure the redirect URI sent in the request matches one added to your application in the Azure portal. Navigate to https://aka.ms/redirectUriMismatchError to learn more about how to fix this.

---

We will work to achieve **delegated access** or access on behalf of a user.

In [ ]:
from msal import ConfidentialClientApplication

In [ ]:
app = ConfidentialClientApplication(app_id, authority='https://login.microsoftonline.com/common',
                                    client_credential=client_secret)

In [ ]:
accounts = app.get_accounts() # Check the cache to see if we have some accounts

In [ ]:
accounts

In [ ]:
app.

For some reason, acquire_token_iteractive isn't available there.  Let's try it with a public application instead of a confidential one.

In [ ]:
from msal import PublicClientApplication

In [ ]:
app = PublicClientApplication(app_id, authority='https://login.microsoftonline.com/consumers')

In [ ]:
result = app.acquire_token_interactive(scopes=['User.Read'])

Result was:
> AADSTS700016: Application with identifier '52e677b3-b988-4573-b9d0-18cdb5c7c67b' was not found in the directory 'Microsoft'. This can happen if the application has not been installed by the administrator of the tenant or consented to by any user in the tenant. You may have sent your authentication request to the wrong tenant.

I got that before, then I switched the authority to be `/consumers` instead of `/common`.  Then I re-ran the code and got this error:

> unauthorized_client: The client does not exist or is not enabled for consumers. If you are the application developer, configure a new application through the App Registrations in the Azure Portal at https://go.microsoft.com/fwlink/?linkid=2083908.

---

Graph API explorer, this is cool [Graph API Explorer](https://developer.microsoft.com/en-us/graph/graph-explorer)

## Temporary Token
I'm having a lot of trouble making this API work,  but the problem is somewhere in how I'm getting my token. I know this because I am able to successfully use [Graph API Explorer](https://developer.microsoft.com/en-us/graph/graph-explorer) to explore my OneNote data, even without having a Sharepoint Online license.  So I'm going to work around this by using the token I got from Graph API Explorer for now, and come back later and figure out how to properly get the token.  I got this token using graph explorer, authorized it for Note.Read, and we're off to the races, hopefully.

In [38]:
access_token = secrets.get_secret('danu-identity.temporary_token')

## OneNote Abstraction

I can see by exploring Graph explorer that working with this data is going to be an adventure.  To make accessing this easier, let's make our own abstraction.

In [55]:
from dataclasses import dataclass

In [56]:
@dataclass
class Notebook:
    name: str
    url: str


In [65]:
class OneNoteClient:
    def __init__(self, access_token: str):
        self._token = access_token
    
    def list_notebooks(self):
        notebooks = []
        resp = self._request('/me/onenote/notebooks')
        value_list = resp['value']
        for notebook_json in value_list:
            url = notebook_json['self']
            name = notebook_json['displayName']
            notebook = Notebook(name=name, url=url)
            notebooks.append(notebook)
        return notebooks
    
    def _request(self, url):
        headers = {'Authorization': f'Bearer {self._token}'}
        response = requests.get('https://graph.microsoft.com/v1.0' + url, headers=headers)
        if response.status_code != 200:
            raise Exception(f'OneNoteClient got {response.status_code} {response.content}')
        return response.json()

one_note = OneNoteClient(access_token)

In [64]:
resp = one_note.list_notebooks()


Exception: OneNoteClient got 400 b'{"error":{"code":"20127","message":"Unknown property name: \'title\'.","innerError":{"date":"2023-10-15T01:40:17","request-id":"36498339-541d-4e0c-ac2e-833b9c14a1a3","client-request-id":"36498339-541d-4e0c-ac2e-833b9c14a1a3"}}}'

In [60]:
resp

[Notebook(name='Argon Collab', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!513'),
 Notebook(name='Casa de Lack', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!213'),
 Notebook(name='Edge4x4-TABLET-T4MQ0ET0', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!413'),
 Notebook(name='GuildWebsite', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!11498'),
 Notebook(name='Kuddles', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-9FA50C10908E5E6C!4057'),
 Notebook(name='Lubritech', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!64100'),
 Notebook(name='Mossms', url='https://graph.microsoft.com/v1.0/users/richardlack@hotmail.com/onenote/notebooks/0-D837C5F832A7D046!196'